In [8]:
from logging import basicConfig
from threading import Thread
from time import sleep
from kazoo.client import KazooClient
import random
import sys

In [9]:
ACTION_COMMIT =   b'commit'
ACTION_ROLLBACK = b'rollback'
WAIT_TIME = 5

In [10]:
basicConfig()

In [11]:
class Client(Thread):
  def __init__(self, root: str, id: int):
    super().__init__()
    self.url = f"{root}/{id}"
    self.root = root
    self.id = id

  def run(self):
        client = KazooClient()
        client.start()

        value = ACTION_COMMIT if random.random() > 0.5 else ACTION_ROLLBACK
        print(f"Client {self.id} request {value.decode()}")
        
        client.create(self.url, value, ephemeral=True)
        
        @client.DataWatch(self.url)
        def watch_self(data, stat):
            if stat.version > 0:
                print(f"Client {self.id} do {data.decode()}")

        sleep(WAIT_TIME)
        client.stop()
        client.close()

In [12]:
class Coordinator():
    def main(self):
        coord = KazooClient()
        coord.start()

        if coord.exists("/task2"):
            coord.delete("/task2", recursive=True)

        coord.create("/task2")
        coord.create("/task2/transaction")

        n_clients = 5

        def solution():
            clients = coord.get_children("/task2/transaction")
            commits, rollbacks = 0, 0
            
            for clt in clients:
                commits += int(coord.get(f"/task2/transaction/{clt}")[0] == ACTION_COMMIT)
                rollbacks += int(coord.get(f"/task2/transaction/{clt}")[0] == ACTION_ROLLBACK)

            for clt in clients:
                coord.set(f"/task2/transaction/{clt}", b"commit" if commits > rollbacks else ACTION_ROLLBACK)

        @coord.ChildrenWatch("/task2/transaction")
        def check_clients(clients):
            if len(clients) < n_clients:
                print(f"Waiting others clients: {clients}")

            elif len(clients) == n_clients:
                print("Check clients")
                solution()

        for x in range(5):
            begin=Client("/task2/transaction", x)
            begin.start()


In [13]:
Coordinator().main()

Waiting others clients: []
Client 0 request rollback
Client 2 request rollback
Client 1 request rollback
Client 4 request rollback
Client 3 request rollback
Check clients
Client 0 do rollback
Client 1 do rollback
Client 2 do rollback
Client 3 do rollback
Client 4 do rollback
Waiting others clients: []
